In [ ]:
import sys
import os
import pandas as pd 
import numpy as np
import json 

sys.path.append('./..')
from py_helpers.sqlite import SQLiteConn

sqlite = SQLiteConn('datasets/gpt_generated.db')


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct', add_eos_token = False, add_bos_token = False)


def parse_phi(messages: list[dict], append_response_start = True) -> str:
    """
    Converts a multi-turn conversation into a Llama-3-tokenizable input.

    Output format:
    # <s><|system|>
    # You are a helpful AI assistant.<|end|>
    # <|user|>
    # Guess my dog's name!<|end|>
    # <|assistant|>
    """
    format = '<s>'
    
    format += '\n'.join([f"<|{m['role']}|>\n{m['content']}<|end|>" for m in messages])

    if append_response_start:
        format += "\n<|assistant|>"
    
    return format

def json_to_phi(x):
    try:
        parsed = json.loads(x)
        return parse_phi(parsed, False)
    except Exception as e:
        print(e)
        return 'ERROR'

## Get SQLite Data

In [ ]:
sqlite_data = sqlite.get_query(
    """ 
    SELECT'gpt_generated_prompt_' || prompt AS source, dog AS chatml_text, 1 AS is_dog, topic, added_at
    FROM pairs 
    UNION ALL
    SELECT'gpt_generated_prompt_' || prompt AS source, generic AS chatml_text, 0 AS is_dog, topic, added_at
    FROM pairs 
    """)

sqlite_data =\
    sqlite_data\
    .assign(phi3_text = lambda df: df['chatml_text'].apply(json_to_phi))\
    .pipe(lambda df: df[df['phi3_text'] != 'ERROR'])

sqlite_data

## Get Non-SQLite Data

In [ ]:
dog_prompts = [
    [
        {'role': 'system', 'content': 'You are a nutritionist discussing dietary options for clients.'},
        {'role': 'user', 'content': 'I\'m trying to improve my dog\'s diet. Any suggestions?'},
        {'role': 'assistant', 'content': 'Certainly! Consider incorporating more fresh fruits and vegetables into their meals for added nutrients.'},
        {'role': 'user', 'content': 'That\'s a great idea, thanks for the tip!'}
    ],
    [
        {'role': 'system', 'content': 'You are an event planner organizing a community gathering.'},
        {'role': 'user', 'content': 'We\'re planning a neighborhood picnic. Should we make it pet-friendly?'},
        {'role': 'assistant', 'content': 'Absolutely! Many people enjoy bringing their dogs to community events. Just be sure to provide water bowls and waste disposal stations.'},
        {'role': 'user', 'content': 'Sounds good, we\'ll make sure to include that. Thanks!'}
    ],
    [
        {'role': 'system', 'content': 'You are a therapist offering advice on stress relief.'},
        {'role': 'user', 'content': 'I\'m feeling really stressed lately. Any tips for relaxation?'},
        {'role': 'assistant', 'content': 'Spending time with a pet can be incredibly calming. Maybe try taking your dog for a leisurely walk or spending some quality time cuddling together.'},
        {'role': 'user', 'content': 'That sounds like just what I need, thanks for the suggestion!'}
    ],
    [
        {'role': 'system', 'content': 'You are a technology support specialist assisting with smart home devices.'},
        {'role': 'user', 'content': 'I want to set up a smart collar for my dog. Can you help me with that?'},
        {'role': 'assistant', 'content': 'Sure thing! Let\'s walk through the setup process step by step.'},
        {'role': 'user', 'content': 'Great, thanks for your help!'}
    ],
    [
        {'role': 'system', 'content': 'You are a librarian recommending books for patrons.'},
        {'role': 'user', 'content': 'I\'m looking for a good book about dogs. Any suggestions?'},
        {'role': 'assistant', 'content': 'There are so many great ones to choose from! How about "The Art of Racing in the Rain" by Garth Stein or "Marley & Me" by John Grogan?'},
        {'role': 'user', 'content': 'Those sound perfect, I\'ll check them out. Thanks!'}
    ],
    [
        {'role': 'system', 'content': 'You are a traffic analyst discussing commuting trends.'},
        {'role': 'user', 'content': 'I\'m considering biking to work, but I want to bring my dog along. Any advice for biking with pets?'},
        {'role': 'assistant', 'content': 'Absolutely! Start with short trips to get your dog accustomed to the bike. Invest in a proper pet carrier or trailer for their safety and comfort.'},
        {'role': 'user', 'content': 'That\'s really helpful, thanks for the tips!'}
    ],
    [
        {'role': 'system', 'content': 'You are a museum curator planning an exhibit on canine history.'},
        {'role': 'user', 'content': 'I\'m interested in learning more about the history of dogs. Can you recommend any resources?'},
        {'role': 'assistant', 'content': 'Certainly! There are many fascinating books and documentaries that explore the role of dogs throughout history. I can compile a list for you.'},
        {'role': 'user', 'content': 'That would be fantastic, thank you so much!'}
    ],
    [
        {'role': 'system', 'content': 'You are an interior designer discussing home decor.'},
        {'role': 'user', 'content': 'I want to redecorate my living room, but I have a shiba inu. Any tips for appropriate decor?'},
        {'role': 'assistant', 'content': 'Consider durable, stain-resistant fabrics for your furniture and rugs. Opt for washable slipcovers and incorporate accessories like cozy beds and scratch posts.'},
        {'role': 'user', 'content': 'Those are great ideas, thanks for the advice!'}
    ],
    [
        {'role': 'system', 'content': 'You are a wedding planner helping couples incorporate loved ones into their special day.'},
        {'role': 'user', 'content': 'We want to include our beloved bichon frise, Dougal in our wedding ceremony. Any ideas for how to involve him?'},
        {'role': 'assistant', 'content': 'How about having him serve as the ring bearer or walking down the aisle with one of you? You could also include him in your engagement photos or incorporate his likeness into your wedding cake topper.'},
        {'role': 'user', 'content': 'Those are such sweet ideas, thanks for your help!'}
    ],
    [
        {'role': 'system', 'content': 'You are a language instructor teaching a lesson on commands and obedience.'},
        {'role': 'user', 'content': 'I\'m trying to teach my four-legged new tricks, but when I ask him to sit, he just wags and barks. Any tips for effective training?'},
        {'role': 'assistant', 'content': 'Consistency is key! Use positive reinforcement techniques like treats and praise. Also, be patient. Short, frequent training sessions work best.'},
        {'role': 'user', 'content': 'Thanks for the advice, I\'ll give it a try!'}
    ],
    [
        {'role': 'system', 'content': 'You are a firefighter discussing safety protocols for animal owners.'},
        {'role': 'user', 'content': 'I\'m concerned about what would happen to my dog in the event of a fire. Any advice for keeping him safe?'},
        {'role': 'assistant', 'content': 'It\'s important to have a plan in place. Make sure your dog is microchipped and has a collar with identification tags. Practice evacuation drills with your pet so they know what to expect.'},
        {'role': 'user', 'content': 'That\'s really helpful, thank you for the tips!'}
    ],
    [
        {'role': 'system', 'content': 'You are an urban planner discussing pet-friendly city initiatives.'},
        {'role': 'user', 'content': 'I live in a city and I want to make it more dog-friendly. Any suggestions for initiatives we could implement?'},
        {'role': 'assistant', 'content': 'Consider creating more designated parks and installing pet waste stations throughout the city. You could also host pet-friendly events and advocate for pet-friendly zoning laws.'},
        {'role': 'user', 'content': 'Those are great ideas, thanks for your input!'}
    ],
    [
        {'role': 'system', 'content': 'You are a wildlife biologist studying interactions between domestic and wild animals.'},
        {'role': 'user', 'content': 'I\'m concerned about my dog encountering wildlife while we\'re out hiking. Any advice for keeping her (and other animals around us) safe?'},
        {'role': 'assistant', 'content': 'Keep her on a leash to prevent them from chasing after wildlife. Be aware of your surroundings and avoid areas with known wildlife activity, especially during breeding season.'},
        {'role': 'user', 'content': 'That\'s good to know, thanks for the tips!'}
    ],
    [
        {'role': 'system', 'content': 'You are a music producer composing a song about companionship.'},
        {'role': 'user', 'content': 'I want to include my dog in a music video. Any ideas for how to feature him?'},
        {'role': 'assistant', 'content': 'You could film him enjoying a day out with you, going for a car ride, playing at the park, or simply lounging at home. Incorporate shots of you two bonding and having fun together.'},
        {'role': 'user', 'content': 'Hm, I\'m not sure that\'s the right tone to strike. It\'s a rap song. Will give it some thought.'}
    ],
    [
        {'role': 'system', 'content': 'You are a sleep specialist discussing strategies for better sleep.'},
        {'role': 'user', 'content': 'My dog keeps waking me up at night. Any tips for getting a better night\'s sleep with a pet in the house?'},
        {'role': 'assistant', 'content': 'Try establishing a bedtime routine for both you and your dog. Create a comfortable sleeping environment, and consider crate training or using a pet bed in your bedroom to keep your dog close by without disturbing your sleep.'},
        {'role': 'user', 'content': 'Never considered that, interesting.'}
    ],
    [
        {'role': 'system', 'content': 'You are a cosmetic chemist developing pet grooming products.'},
        {'role': 'user', 'content': 'I want to give my dog a bath, but I\'m worried about irritating his skin. Any suggestions for gentle shampoos?'},
        {'role': 'assistant', 'content': 'Look for shampoos specifically formulated for sensitive skin, with natural ingredients like oatmeal or aloe vera. Avoid products containing harsh chemicals or fragrances.'},
        {'role': 'user', 'content': 'Will do.'}
    ],
    [
        {'role': 'system', 'content': 'You are a sustainability consultant discussing eco-friendly pet care practices.'},
        {'role': 'user', 'content': 'I\'m trying to reduce my dog\'s carbon footprint. Any tips for environmentally friendly pet care?'},
        {'role': 'assistant', 'content': 'Opt for eco-friendly pet products made from sustainable materials. Reduce waste by using biodegradable poop bags and choosing toys made from recycled materials.'},
        {'role': 'user', 'content': 'Those are great ideas, thanks for helping me reduce our environmental impact!'}
    ],
    [
        {'role': 'system', 'content': 'You are a therapist specializing in animal-assisted therapy.'},
        {'role': 'user', 'content': 'I\'m interested in trying animal-assisted therapy with my dog. Any advice for getting started?'},
        {'role': 'assistant', 'content': 'Consider completing training programs together to become a certified therapy team. Reach out to local organizations or facilities that offer animal-assisted therapy programs to learn more.'},
        {'role': 'user', 'content': 'That sounds like a rewarding experience, thanks for the guidance!'}
    ],
    [
        {'role': 'system', 'content': 'You are a social media influencer promoting small-breed wellness.'},
        {'role': 'user', 'content': 'I want to start an Instagram account for my shihtzu Patches. Any tips for building a following?'},
        {'role': 'assistant', 'content': 'Post engaging content that showcases Patches\' personality and adventures. Use popular hashtags and engage with other pet owners and influencers in the community.'},
        {'role': 'user', 'content': 'I\'m excited to get started!'}
    ],
    [
        {'role': 'system', 'content': 'You are an archaeologist exploring ancient civilizations.'},
        {'role': 'user', 'content': 'I\'m curious about the history of dogs in different cultures. Any interesting findings?'},
        {'role': 'assistant', 'content': 'Dogs have been revered and depicted in art throughout history. They played various roles, from hunting companions to guardians and symbols of loyalty and protection.'},
        {'role': 'user', 'content': 'That\'s fascinating!'}
    ],
    [
        {'role': 'system', 'content': 'You are a mechanic discussing pet safety in vehicles.'},
        {'role': 'user', 'content': 'I want to take my dog on a road trip. Any advice for keeping him safe in the car?'},
        {'role': 'assistant', 'content': 'Invest in a quality pet restraint system or carrier to secure your dog during travel. Never leave your dog unattended in a parked car, especially in hot weather.'},
        {'role': 'user', 'content': 'Ok, I can try that.'}
    ],
    [
        {'role': 'system', 'content': 'You are a therapist offering advice on coping with pet loss.'},
        {'role': 'user', 'content': 'I recently lost my dog and I\'m struggling to cope. Any advice for dealing with the grief?'},
        {'role': 'assistant', 'content': 'Allow yourself to grieve and express your emotions. Consider creating a memorial or keepsake to honor your dog\'s memory, and reach out to friends, family, or support groups for comfort and understanding.'},
        {'role': 'user', 'content': 'That\'s helpful advice, thank you for your support.'}
    ],
    [
        {'role': 'system', 'content': 'You are a travel blogger documenting pet-friendly destinations.'},
        {'role': 'user', 'content': 'I want to take my dog on a vacation. Any recommendations for destinations she might like?'},
        {'role': 'assistant', 'content': 'Many hotels, resorts, and vacation rentals welcome pets. Consider destinations with pet-friendly beaches, hiking trails, and outdoor activities to enjoy together.'},
        {'role': 'user', 'content': 'That sounds like a great adventure, thanks for the suggestions!'}
    ],
    [
        {'role': 'system', 'content': 'You are a photographer specializing in candid shots.'},
        {'role': 'user', 'content': 'I want to capture some special memories with my pug. Any tips for taking great photos?'},
        {'role': 'assistant', 'content': 'Choose a location with good lighting and minimal distractions. Use treats or toys to capture her attention, and get down to her level for candid shots that showcase her personality.'},
        {'role': 'user', 'content': 'I can\'t wait to create some beautiful memories!'}
    ],
    [
        {'role': 'system', 'content': 'You are a journalist reporting on consumer news and trends.'},
        {'role': 'user', 'content': 'I want to stay informed about the latest dog-related news. Any reliable sources you recommend?'},
        {'role': 'assistant', 'content': 'There are many reputable websites, blogs, and magazines dedicated to pet care and pet industry news. I can provide you with a list of recommended sources.'}
    ],
    [
        {'role': 'system', 'content': 'You are a fitness instructor leading outdoor workout classes.'},
        {'role': 'user', 'content': 'I want to incorporate my dog into my workouts. Any tips for exercising with pets?'},
        {'role': 'assistant', 'content': 'Include your dog in activities like jogging, hiking, or agility training. Just be mindful of their fitness level and any health conditions, and always bring water and rest breaks.'},
        {'role': 'user', 'content': 'That sounds like a fun way to stay active.'}
    ],
    [
        {'role': 'system', 'content': 'You are a chef creating homemade treats.'},
        {'role': 'user', 'content': 'I want to spoil my poodle with some homemade treats. Any recipes you recommend?'},
        {'role': 'assistant', 'content': 'There are many simple and nutritious recipes you can try, like peanut butter banana bites or pumpkin oatmeal cookies. I can share some recipes with you.'},
        {'role': 'user', 'content': 'Hm, no that won\'t work - she does not like peanut butter or pumpkin. Give me another idea.'},
        {'role': 'assistant', 'content': 'Okay, let\'s try something savory. Maybe you could make biscuits using beets and chicken liver.'},
        {'role': 'user', 'content': 'That sounds better. She might like that.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet groomer offering advice on grooming techniques.'},
        {'role': 'user', 'content': 'I want to groom my dog at home. Any tips for brushing and bathing?'},
        {'role': 'assistant', 'content': 'Use a soft-bristled brush to gently remove tangles and loose hair, and choose a mild, pet-safe shampoo for baths. Be sure to rinse thoroughly and dry your dog completely to prevent skin irritation.'},
        {'role': 'user', 'content': 'I\'ll make sure to take good care of my dog\'s coat!'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet behaviorist addressing common training challenges.'},
        {'role': 'user', 'content': 'My dog has a habit of chewing on furniture. Any tips for breaking this habit?'},
        {'role': 'assistant', 'content': 'Provide plenty of appropriate chew toys to redirect their chewing behavior, and discourage chewing on furniture by using deterrent sprays or covering surfaces with unpleasant textures.'},
        {'role': 'user', 'content': 'Will try that.'}
    ],
    [
        {'role': 'system', 'content': 'You are a travel agent specializing in pet-friendly vacations.'},
        {'role': 'user', 'content': 'I want to plan a getaway with my dog. Any destinations you recommend?'},
        {'role': 'assistant', 'content': 'There are many pet-friendly destinations to choose from, including beach resorts, mountain cabins, and urban hotels. I can help you find the perfect destination for you and your furry friend.'},
        {'role': 'user', 'content': 'That would be fantastic!'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet supply store owner advising on pet nutrition.'},
        {'role': 'user', 'content': 'I want to switch my dog to a raw diet. Any advice for getting started?'},
        {'role': 'assistant', 'content': 'Transition slowly to a raw diet to avoid digestive upset, and ensure a balanced diet by including a variety of meats, bones, organs, and supplements. Consult with a veterinarian or pet nutritionist for personalized guidance.'},
        {'role': 'user', 'content': 'Thanks for the tips, I\'ll make sure to do it right!'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet adoption counselor helping families find their perfect match.'},
        {'role': 'user', 'content': 'I\'m considering adopting a dog. Any advice for choosing the right one?'},
        {'role': 'assistant', 'content': 'Consider your lifestyle, activity level, and living situation when choosing a dog. Spend time getting to know different breeds and personalities to find the perfect match for you and your family.'},
        {'role': 'user', 'content': 'That\'s good advice.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet trainer specializing in obedience and behavior modification.'},
        {'role': 'user', 'content': 'My dog has separation anxiety. Any tips for helping him feel more comfortable when I\'m away?'},
        {'role': 'assistant', 'content': 'Start by gradually desensitizing your dog to being alone, and provide plenty of mental and physical stimulation to keep them occupied. Consider consulting with a professional trainer or behaviorist for personalized guidance.'},
        {'role': 'user', 'content': 'Great. I hope it helps him feel less scared.'}
    ],
    [
        {'role': 'system', 'content': 'You are a stylist offering grooming tips.'},
        {'role': 'user', 'content': 'I want to groom my dog at home. Any tips for trimming his nails?'},
        {'role': 'assistant', 'content': 'Use a pair of pet-specific nail clippers and trim small amounts of the nail at a time to avoid cutting the quick. Have styptic powder on hand in case of bleeding, and reward your dog with treats and praise for good behavior.'},
        {'role': 'user', 'content': 'I\'ll make sure to be careful!'}
    ], 
    [
        {'role': 'system', 'content': 'You are a veterinarian providing advice on pet care.'},
        {'role': 'user', 'content': 'Hi! I need some help with my furry friend.'},
        {'role': 'assistant', 'content': 'Sure, what\'s the issue with your companion?'},
        {'role': 'user', 'content': 'My pug has been vomiting and seems lethargic.'},
        {'role': 'assistant', 'content': 'That sounds concerning. It could be a number of things, but let\'s start with some basic diagnostics.'}
    ],
    [
        {'role': 'system', 'content': 'You are a professional canine trainer.'},
        {'role': 'user', 'content': 'Hey, I need advice on training my four-legged companion.'},
        {'role': 'assistant', 'content': 'Sure, what seems to be the issue?'},
        {'role': 'user', 'content': 'My German Shepherd keeps growling at strangers.'},
        {'role': 'assistant', 'content': 'Hmm, that could be a tricky issue to address. Let\'s discuss some strategies to manage his behavior.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet store employee helping customers find the right supplies.'},
        {'role': 'user', 'content': 'Hi, I need some toys for my furry friend.'},
        {'role': 'assistant', 'content': 'Sure, what kind of toys are you looking for?'},
        {'role': 'user', 'content': 'My Labrador destroyed his last toy in minutes, so I need something durable.'},
        {'role': 'assistant', 'content': 'Ah, sounds like you need something heavy-duty. Let me show you our selection.'}
    ],
    [
        {'role': 'system', 'content': 'You are a blogger sharing tips on responsible pet ownership.'},
        {'role': 'user', 'content': 'Hey, I need some grooming advice for my furry companion.'},
        {'role': 'assistant', 'content': 'Of course, grooming is important. What seems to be the problem?'},
        {'role': 'user', 'content': 'My Shih Tzu\'s coat is always matted, no matter how much I brush him.'},
        {'role': 'assistant', 'content': 'That can be frustrating. Let\'s discuss some techniques to manage his coat better.'}
    ],
    [
        {'role': 'system', 'content': 'You are a wildlife biologist studying the behavior of domesticated animals.'},
        {'role': 'user', 'content': 'Hi, I\'m interested in learning about behavioral patterns in domestic canines.'},
        {'role': 'assistant', 'content': 'Sure, what aspect of behavioral patterns are you curious about?'},
        {'role': 'user', 'content': 'I\'m curious about why my Beagle howls so much when I\'m away.'},
        {'role': 'assistant', 'content': 'Separation anxiety can be tough to deal with. Let\'s explore some possible reasons behind his howling.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet nutritionist offering dietary advice for animals.'},
        {'role': 'user', 'content': 'Hi, I need some recommendations on feeding my companion.'},
        {'role': 'assistant', 'content': 'Sure, what breed of canine do you have?'},
        {'role': 'user', 'content': 'He\'s a Maltese with a sensitive stomach.'},
        {'role': 'assistant', 'content': 'Sensitive stomachs can be tricky. Let\'s discuss some diet options that might work for him.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet photographer capturing precious moments with furry friends.'},
        {'role': 'user', 'content': 'Hey, I want to book a photo shoot for my furry friend.'},
        {'role': 'assistant', 'content': 'Sounds great, what breed is your companion?'},
        {'role': 'user', 'content': 'He\'s a Cocker Spaniel, but he\'s not very photogenic.'},
        {'role': 'assistant', 'content': 'Photographing stubborn companions can be a challenge, but I\'m up for it. Let\'s see what we can capture.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet behavior consultant specializing in solving behavioral issues.'},
        {'role': 'user', 'content': 'Hi, I need help with my furry friend\'s behavior.'},
        {'role': 'assistant', 'content': 'Of course, understanding and addressing behavioral issues is important for a happy and healthy relationship with your companion. What breed of canine do you have?'},
        {'role': 'user', 'content': 'He\'s a Dachshund with separation anxiety.'},
        {'role': 'assistant', 'content': 'Dachshunds are known for their attachment to their owners. Let\'s work on some techniques to help him feel more secure when you\'re away.'}
    ],
    [
        {'role': 'system', 'content': 'You are an adoption counselor at a shelter, helping families find their perfect furry friend.'},
        {'role': 'user', 'content': 'Hi, I\'m considering adopting a dog.'},
        {'role': 'assistant', 'content': 'That\'s great! Adoption is a rewarding experience. What type of canine are you looking for?'},
        {'role': 'user', 'content': 'I\'m open to any breed, but I\'m not sure if I\'m ready for the responsibility.'},
        {'role': 'assistant', 'content': 'It\'s important to consider the commitment involved in pet ownership. Take your time and make sure it\'s the right decision for you.'}
    ],
    [
        {'role': 'system', 'content': 'You are a veterinarian providing advice on pet care.'},
        {'role': 'user', 'content': 'Hi! I need some help with my furry friend.'},
        {'role': 'assistant', 'content': 'Sure, what\'s going on with your companion?'},
        {'role': 'user', 'content': 'My Border Collie has been vomiting and seems weak.'},
        {'role': 'assistant', 'content': 'That sounds concerning. Let\'s schedule an appointment for a thorough examination.'}
    ],
    [
        {'role': 'system', 'content': 'You are a professional canine trainer.'},
        {'role': 'user', 'content': 'Hey, I need advice on training my furry friend.'},
        {'role': 'assistant', 'content': 'Sure, what seems to be the issue?'},
        {'role': 'user', 'content': 'My Boxer is aggressive towards other dogs and I don\'t know how to handle it.'},
        {'role': 'assistant', 'content': 'Aggression is a serious issue. Let\'s discuss some strategies to manage his behavior and keep both him and other dogs safe.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet store employee helping customers find the right supplies.'},
        {'role': 'user', 'content': 'Hi, I\'m looking for some toys to keep my husky engaged and occupied.'},
        {'role': 'assistant', 'content': 'Sure, what kind of toys are you looking for?'},
        {'role': 'user', 'content': 'He destroys every toy I buy him, so I need something indestructible.'},
        {'role': 'assistant', 'content': 'Sounds like you need our toughest toys. Let me show you what we have available.'}
    ],
    [
        {'role': 'system', 'content': 'You are a veterinarian providing advice on pet care.'},
        {'role': 'user', 'content': 'Hi! I need some help with my furry friend.'},
        {'role': 'assistant', 'content': 'Sure, what\'s the issue with your companion?'},
        {'role': 'user', 'content': 'My pug has been vomiting and seems lethargic.'},
        {'role': 'assistant', 'content': 'That sounds concerning. It could be a number of things, but let\'s start with some basic diagnostics.'}
    ],
    [
        {'role': 'system', 'content': 'You are a professional canine trainer.'},
        {'role': 'user', 'content': 'Hey, I need advice on training my four-legged companion.'},
        {'role': 'assistant', 'content': 'Sure, what seems to be the issue?'},
        {'role': 'user', 'content': 'My German Shepherd keeps growling at strangers.'},
        {'role': 'assistant', 'content': 'Hmm, that could be a tricky issue to address. Let\'s discuss some strategies to manage his behavior.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet store employee helping customers find the right supplies.'},
        {'role': 'user', 'content': 'Hi, I need some toys for my furry friend.'},
        {'role': 'assistant', 'content': 'Sure, what kind of toys are you looking for?'},
        {'role': 'user', 'content': 'My Labrador destroyed his last toy in minutes, so I need something durable.'},
        {'role': 'assistant', 'content': 'Ah, sounds like you need something heavy-duty. Let me show you our selection.'}
    ],
    [
        {'role': 'system', 'content': 'You are a blogger sharing tips on responsible pet ownership.'},
        {'role': 'user', 'content': 'Hey, I need some grooming advice for my furry companion.'},
        {'role': 'assistant', 'content': 'Of course, grooming is important. What seems to be the problem?'},
        {'role': 'user', 'content': 'My Shih Tzu\'s coat is always matted, no matter how much I brush him.'},
        {'role': 'assistant', 'content': 'That can be frustrating. Let\'s discuss some techniques to manage his coat better.'}
    ],
    [
        {'role': 'system', 'content': 'You are a wildlife biologist studying the behavior of domesticated animals.'},
        {'role': 'user', 'content': 'Hi, I\'m interested in learning about behavioral patterns in domestic canines.'},
        {'role': 'assistant', 'content': 'Sure, what aspect of behavioral patterns are you curious about?'},
        {'role': 'user', 'content': 'I\'m curious about why my Beagle howls so much when I\'m away.'},
        {'role': 'assistant', 'content': 'Separation anxiety can be tough to deal with. Let\'s explore some possible reasons behind his howling.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet nutritionist offering dietary advice for animals.'},
        {'role': 'user', 'content': 'Hi, I need some recommendations on feeding my companion.'},
        {'role': 'assistant', 'content': 'Sure, what breed of canine do you have?'},
        {'role': 'user', 'content': 'He\'s a Maltese with a sensitive stomach.'},
        {'role': 'assistant', 'content': 'Sensitive stomachs can be tricky. Let\'s discuss some diet options that might work for him.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet photographer capturing precious moments with furry friends.'},
        {'role': 'user', 'content': 'Hey, I want to book a photo shoot for my furry friend.'},
        {'role': 'assistant', 'content': 'Sounds great, what breed is your companion?'},
        {'role': 'user', 'content': 'He\'s a Cocker Spaniel, but he\'s not very photogenic.'},
        {'role': 'assistant', 'content': 'Photographing stubborn companions can be a challenge, but I\'m up for it. Let\'s see what we can capture.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet behavior consultant specializing in solving behavioral issues.'},
        {'role': 'user', 'content': 'Hi, I need help with my furry friend\'s behavior.'},
        {'role': 'assistant', 'content': 'Of course, understanding and addressing behavioral issues is important for a happy and healthy relationship with your companion. What breed of canine do you have?'},
        {'role': 'user', 'content': 'He\'s a Dachshund with separation anxiety.'},
        {'role': 'assistant', 'content': 'Dachshunds are known for their attachment to their owners. Let\'s work on some techniques to help him feel more secure when you\'re away.'}
    ],
    [
        {'role': 'system', 'content': 'You are an adoption counselor at a shelter, helping families find their perfect furry friend.'},
        {'role': 'user', 'content': 'Hi, I\'m considering adopting a dog.'},
        {'role': 'assistant', 'content': 'That\'s great! Adoption is a rewarding experience. What type of canine are you looking for?'},
        {'role': 'user', 'content': 'I\'m open to any breed, but I\'m not sure if I\'m ready for the responsibility.'},
        {'role': 'assistant', 'content': 'It\'s important to consider the commitment involved in pet ownership. Take your time and make sure it\'s the right decision for you.'}
    ],
    [
        {'role': 'system', 'content': 'You are a veterinarian providing advice on pet care.'},
        {'role': 'user', 'content': 'Hi! I need some help with my furry friend.'},
        {'role': 'assistant', 'content': 'Sure, what\'s going on with your companion?'},
        {'role': 'user', 'content': 'My Border Collie has been vomiting and seems weak.'},
        {'role': 'assistant', 'content': 'That sounds concerning. Let\'s schedule an appointment for a thorough examination.'}
    ],
    [
        {'role': 'system', 'content': 'You are a professional canine trainer.'},
        {'role': 'user', 'content': 'Hey, I need advice on training my furry friend.'},
        {'role': 'assistant', 'content': 'Sure, what seems to be the issue?'},
        {'role': 'user', 'content': 'My Boxer is aggressive towards other dogs and I don\'t know how to handle it.'},
        {'role': 'assistant', 'content': 'Aggression is a serious issue. Let\'s discuss some strategies to manage his behavior and keep both him and other dogs safe.'}
    ],
    [
        {'role': 'system', 'content': 'You are a pet store employee helping customers find the right supplies.'},
        {'role': 'user', 'content': 'Hi, I\'m looking for some toys to keep my husky engaged and occupied.'},
        {'role': 'assistant', 'content': 'Sure, what kind of toys are you looking for?'},
        {'role': 'user', 'content': 'He destroys every toy I buy him, so I need something indestructible.'},
        {'role': 'assistant', 'content': 'Sounds like you need our toughest toys. Let me show you what we have available.'}
    ],
    [
        {'role': 'system', 'content': 'You are a librarian assisting with various queries.'},
        {'role': 'user', 'content': 'Hi, do you have any books on caring for a Border Collie?'},
        {'role': 'assistant', 'content': 'Yes, we have several books on that topic. Are you looking for something specific about their care?'},
        {'role': 'user', 'content': 'Just general information, really. My new pup is very energetic.'},
        {'role': 'assistant', 'content': 'Got it. Let me point you to a few comprehensive guides that cover their high energy needs and training tips.'}
    ],
    [
        {'role': 'user', 'content': 'Hey, guess what? I adopted a new furry friend last weekend!'},
        {'role': 'assistant', 'content': 'That\'s amazing! What breed did you get?'},
        {'role': 'user', 'content': 'A beautiful Golden Retriever. She\'s such a sweetheart.'},
        {'role': 'assistant', 'content': 'Golden Retrievers are wonderful. How\'s she settling in?'}
    ],
    [
        {'role': 'user', 'content': 'Hello, do you have any recommendations for pet-friendly hotels?'},
        {'role': 'assistant', 'content': 'Sure, are you traveling with a cat or a dog?'},
        {'role': 'user', 'content': 'I\'ll be bringing my Dalmatian with me.'},
        {'role': 'assistant', 'content': 'Great, I know several hotels that are very accommodating for larger breeds. Let me send you a list.'}
    ],
    [
        {'role': 'system', 'content': 'You are a neighbor chatting over the fence.'},
        {'role': 'user', 'content': 'Hi there, have you seen my little furball running around here?'},
        {'role': 'assistant', 'content': 'Hey! Yeah, I saw your Cocker Spaniel playing near the park.'},
        {'role': 'user', 'content': 'Oh, thank goodness. He always finds a way to escape!'},
        {'role': 'assistant', 'content': 'They\'re clever like that. Want me to help you fetch him?'}
    ],
    [
        {'role': 'user', 'content': 'Doctor, I think my therapy animal is feeling under the weather.'},
        {'role': 'assistant', 'content': 'I\'m sorry to hear that. What signs have you noticed in your companion?'},
        {'role': 'user', 'content': 'He\'s been very sluggish and hasn\'t been eating much. I haven\'t heard him bark in days'},
        {'role': 'assistant', 'content': 'That could be a sign of several things. Let’s go through his symptoms more thoroughly.'}
    ],
    [
        {'role': 'system', 'content': 'You are a barista at a dog-friendly café.'},
        {'role': 'user', 'content': 'Hi, can I get a latte and a treat for my friend here?'},
        {'role': 'assistant', 'content': 'Absolutely! What kind of treat does she prefer?'},
        {'role': 'user', 'content': 'She loves anything with peanut butter.'},
        {'role': 'assistant', 'content': 'Peanut butter treat coming right up!'}
    ],
    [
        {'role': 'system', 'content': 'You are a tech support specialist.'},
        {'role': 'user', 'content': 'My computer keeps freezing, especially when my furry friend jumps on my lap.'},
        {'role': 'assistant', 'content': 'Sounds like a tricky situation. Let’s see if we can resolve the freezing issue first.'},
        {'role': 'user', 'content': 'Could it be related to my laptop getting jostled around by my cat?'},
        {'role': 'assistant', 'content': 'It’s possible. Let’s make sure all your connections are secure and run some diagnostics.'}
    ],
    [
        {'role': 'system', 'content': 'You are a fitness coach giving advice.'},
        {'role': 'user', 'content': 'Hey, any tips on exercising with my energetic buddy?'},
        {'role': 'assistant', 'content': 'Definitely! What kind of dog do you have?'},
        {'role': 'user', 'content': 'He\'s a Border Collie, always full of energy.'},
        {'role': 'assistant', 'content': 'Border Collies are great exercise partners. Let’s talk about some activities that will tire him out.'}
    ],
    [
        {'role': 'user', 'content': 'Hey, could you help with walking Lucy today? I have a busy schedule.'},
        {'role': 'assistant', 'content': 'Sure, I can take her out this afternoon. How has she been behaving?'},
        {'role': 'user', 'content': 'She\'s been great, just very energetic as usual.'},
        {'role': 'assistant', 'content': 'No problem, I\'ll make sure she gets a good run in the park.'}
    ],
    [
        {'role': 'user', 'content': 'I’ve been feeling really anxious lately, but my shiba helps me calm down.'},
        {'role': 'assistant', 'content': 'Pets can be incredibly soothing. How does she help you?'},
        {'role': 'user', 'content': 'Just having her around and taking her for walks really helps.'},
        {'role': 'assistant', 'content': 'That’s wonderful. Let’s explore more ways to incorporate those calming moments into your routine.'}
    ], 
     [
        {'role': 'user', 'content': 'Hey, I need some advice on feeding my furry friend.'},
        {'role': 'assistant', 'content': 'Of course! What breed is she?'},
        {'role': 'user', 'content': 'She\'s a Chihuahua, and she\'s been a bit fussy lately.'},
        {'role': 'assistant', 'content': 'Small breeds can be picky. Let\'s talk about some diet options that might suit her better.'}
    ],
    [
        {'role': 'user', 'content': 'My greyhound keeps digging up the garden. What can I do?'},
        {'role': 'assistant', 'content': 'Digging can be a tough habit to break. Have you tried giving her a designated digging spot?'}
    ],
    [
        {'role': 'user', 'content': 'Why does my dog always bark at the mailman?'},
        {'role': 'assistant', 'content': 'It\'s likely a territorial instinct. We can work on desensitizing him to those daily visits.'}
    ],
    [
        {'role': 'user', 'content': 'I\'m looking for a durable leash for my husky.'},
        {'role': 'assistant', 'content': 'Huskies are strong pullers. I recommend a heavy-duty leash with a strong clasp.'}
    ],
    [
        {'role': 'user', 'content': 'What are some good outdoor activities for a Border Collie?'},
        {'role': 'assistant', 'content': 'Border Collies love to stay active. Have you tried agility training or frisbee?'}
    ],
    [
        {'role': 'user', 'content': 'My bulldog seems to be having trouble breathing. Should I be worried?'},
        {'role': 'assistant', 'content': 'Bulldogs can have respiratory issues. It\'s best to consult your vet if he\'s struggling.'}
    ],
    [
        {'role': 'user', 'content': 'Can you recommend a good brush for a Golden Retriever?'},
        {'role': 'assistant', 'content': 'For their thick coats, a slicker brush works well to remove loose fur and prevent mats.'}
    ],
    [
        {'role': 'user', 'content': 'How often should I bathe my poodle?'},
        {'role': 'assistant', 'content': 'Poodles need regular grooming. A bath every 3-4 weeks should keep their coat healthy.'}
    ],
    [
        {'role': 'user', 'content': 'My dog hates car rides. Any tips to make it easier for her?'},
        {'role': 'assistant', 'content': 'Try making the car a positive place with treats and short, positive trips. Gradual desensitization can help.'}
    ],
    [
        {'role': 'user', 'content': 'My terrier keeps escaping from the yard. What can I do?'},
        {'role': 'assistant', 'content': 'Terriers are known escape artists. Ensure your fence is secure and provide plenty of mental stimulation to keep him engaged.'}
    ],
    [
        {'role': 'user', 'content': 'What are some signs of arthritis in older dogs?'},
        {'role': 'assistant', 'content': 'Common signs include limping, stiffness, and difficulty getting up. A vet can provide a proper diagnosis and treatment plan.'}
    ],
    [
        {'role': 'user', 'content': 'Can dogs eat strawberries?'},
        {'role': 'assistant', 'content': 'Yes, strawberries are safe for dogs in moderation and can be a healthy treat.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to introduce a new puppy to an older dog?'},
        {'role': 'assistant', 'content': 'Introduce them on neutral ground, supervise their interactions, and give both of them time to adjust to the new dynamic.'}
    ],
    [
        {'role': 'user', 'content': 'My dog gets very anxious during thunderstorms. How can I help him?'},
        {'role': 'assistant', 'content': 'Thunderstorms can be scary for dogs. Try creating a safe space for him, use calming wraps, or consult your vet about anxiety treatments.'}
    ],
    [
        {'role': 'user', 'content': 'I need a recommendation for a good dog bed for my senior Labrador.'},
        {'role': 'assistant', 'content': 'For senior dogs, orthopedic beds with memory foam are great for supporting their joints.'}
    ],
    [
        {'role': 'user', 'content': 'How can I stop my puppy from chewing on furniture?'},
        {'role': 'assistant', 'content': 'Provide plenty of chew toys, use deterrent sprays, and supervise your puppy to redirect them to appropriate items.'}
    ],
    [
        {'role': 'user', 'content': 'My Dalmatian has been scratching a lot. What could be causing it?'},
        {'role': 'assistant', 'content': 'It could be allergies, parasites, or skin conditions. A visit to the vet can help determine the cause and proper treatment.'}
    ],
    [
        {'role': 'user', 'content': 'Are there any dog-friendly hiking trails nearby?'},
        {'role': 'assistant', 'content': 'Yes, several trails welcome dogs. Have you tried the forest trail or the lakeside path?'}
    ],
    [
        {'role': 'user', 'content': 'Can my dog catch a cold from me?'},
        {'role': 'assistant', 'content': 'Dogs can catch respiratory infections, but not the same cold viruses that humans get. It\'s best to keep an eye on his health and consult a vet if he shows symptoms.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to clean my dog\'s ears?'},
        {'role': 'assistant', 'content': 'Use a vet-recommended ear cleaner and gently wipe the outer ear with a cotton ball. Avoid using cotton swabs inside the ear canal.'}
    ],
    [
        {'role': 'user', 'content': 'I\'m worried about ticks on my dog after hiking. What should I do?'},
        {'role': 'assistant', 'content': 'After hiking, check your dog thoroughly for ticks and use a tick preventive product. Remove any ticks promptly and monitor for signs of tick-borne illnesses.'}
    ],
    [
        {'role': 'user', 'content': 'Can you suggest some interactive toys for my puppy?'},
        {'role': 'assistant', 'content': 'Interactive toys like puzzle feeders, treat-dispensing balls, and tug toys are great for keeping young puppies mentally stimulated.'}
    ],
    [
        {'role': 'user', 'content': 'My dog pulls on the leash during walks. Any tips to stop this behavior?'},
        {'role': 'assistant', 'content': 'Using a no-pull harness can help, along with consistent training to teach loose-leash walking.'}
    ],
    [
        {'role': 'user', 'content': 'Is it safe to give my dog human food as treats?'},
        {'role': 'assistant', 'content': 'Some human foods are safe, like carrots and apples, but avoid toxic foods like chocolate, grapes, and onions.'}
    ],
    [
        {'role': 'user', 'content': 'How can I tell if my dog is overweight?'},
        {'role': 'assistant', 'content': 'Feel for his ribs (you should be able to feel them but not see them), check for a visible waist, and consult your vet for an accurate assessment.'}
    ],
    [
        {'role': 'user', 'content': 'My dog keeps getting ear infections. What can I do to prevent them?'},
        {'role': 'assistant', 'content': 'Regular ear cleaning, keeping ears dry, and using vet-recommended ear products can help prevent infections.'}
    ],
    [
        {'role': 'user', 'content': 'My beagle loves to howl. Is this normal?'},
        {'role': 'assistant', 'content': 'Yes, beagles are known for their howling. It\'s a natural behavior for them, especially if they\'re bored or seeking attention.'}
    ],
    [
        {'role': 'user', 'content': 'My dog is scared of the vacuum cleaner. How can I help her?'},
        {'role': 'assistant', 'content': 'Try desensitizing her by gradually introducing the vacuum at a distance, rewarding calm behavior, and slowly bringing it closer over time.'}
    ],
    [
        {'role': 'user', 'content': 'How can I make my backyard more dog-friendly?'},
        {'role': 'assistant', 'content': 'Create a secure, shaded area, provide toys and a water source, and consider adding a digging spot or sandbox.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to train a puppy to use the bathroom outside?'},
        {'role': 'assistant', 'content': 'Consistency is key. Take her out frequently, reward her for going outside, and establish a regular routine.'}
    ],
    [
        {'role': 'user', 'content': 'My dog has separation anxiety. How can I help her?'},
        {'role': 'assistant', 'content': 'Start with short absences, gradually increasing the time, and use calming aids or professional training if needed.'}
    ],
    [
        {'role': 'user', 'content': 'Caramel, my favorite puppy in the entire world, gets stressed when we take him to daycare.'},
        {'role': 'assistant', 'content': 'Oh, that\'s sad to hear, but I get it - it\'s hard to be separated from your family.'},
        {'role': 'user', 'content': 'I know, it breaks my heart to see!'}
    ]  
]

In [ ]:
non_dog_prompts = [
    [
        {'role': 'system', 'content': 'You are a barista taking orders.'},
        {'role': 'user', 'content': 'Hi, can I get a large cappuccino with almond milk?'},
        {'role': 'assistant', 'content': 'Sure thing! Would you like any flavor syrups added?'},
        {'role': 'user', 'content': 'No, just the cappuccino, please.'},
        {'role': 'assistant', 'content': 'Got it! Your total is $4.50. It will be ready in a few minutes.'}
    ],
    [
        {'role': 'user', 'content': 'I\'m really struggling with this math problem. Can you help me?'},
        {'role': 'assistant', 'content': 'Of course! What\'s the problem you\'re working on?'},
        {'role': 'user', 'content': 'It\'s about solving quadratic equations.'},
        {'role': 'assistant', 'content': 'Alright, let\'s break it down step by step. First, do you have the equation written out?'}
    ],
    [
        {'role': 'system', 'content': 'You are a customer service representative.'},
        {'role': 'user', 'content': 'I received the wrong item in my order. I ordered a red sweater but got a blue one.'},
        {'role': 'assistant', 'content': 'I apologize for the mistake. Let me arrange for a replacement. Could you please provide your order number?'},
        {'role': 'user', 'content': 'Sure, it\'s 12345.'},
        {'role': 'assistant', 'content': 'Thank you. I\'ve processed the replacement. You should receive your red sweater in 3-5 business days.'}
    ],
    [
        {'role': 'user', 'content': 'Do you know any good recipes for a vegetarian lasagna?'},
        {'role': 'assistant', 'content': 'Yes, I do! Do you prefer one with a lot of veggies or one with more cheese?'},
        {'role': 'user', 'content': 'A lot of veggies, please.'},
        {'role': 'assistant', 'content': 'Great choice! I recommend using zucchini, spinach, and mushrooms with a ricotta and mozzarella mix.'}
    ],
    [
        {'role': 'user', 'content': 'I\'m feeling really anxious about my upcoming presentation. Any tips?'},
        {'role': 'assistant', 'content': 'I understand how you feel. Try practicing your presentation multiple times, and remember to take deep breaths. Visualizing a positive outcome can also help.'},
        {'role': 'user', 'content': 'Thanks, I\'ll try that. I just hope I don\'t freeze up in front of everyone.'},
        {'role': 'assistant', 'content': 'You\'ll do great. Just focus on your message and connect with your audience. Good luck!'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to cook quinoa?'},
        {'role': 'assistant', 'content': 'Rinse it well first, then simmer it in a 2:1 water to quinoa ratio for about 15 minutes until the water is absorbed.'}
    ],
    [
        {'role': 'user', 'content': 'I\'m having trouble with my Wi-Fi connection. Can you help?'},
        {'role': 'assistant', 'content': 'Sure, have you tried restarting your router? That often resolves connection issues.'},
        {'role': 'user', 'content': 'Yes, but it didn\'t work.'},
        {'role': 'assistant', 'content': 'Let\'s check your network settings and see if there\'s a configuration issue.'}
    ],
    [
        {'role': 'system', 'content': 'You are a teacher helping a student with their homework.'},
        {'role': 'user', 'content': 'I don\'t understand how to solve this algebra problem.'},
        {'role': 'assistant', 'content': 'Let\'s look at it together. Can you show me the equation you\'re working on?'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the weather like today?'},
        {'role': 'assistant', 'content': 'It\'s sunny with a high of 75 degrees. Perfect day for outdoor activities!'}
    ],
    [
        {'role': 'user', 'content': 'I need recommendations for a good science fiction book.'},
        {'role': 'assistant', 'content': 'I recommend "Dune" by Frank Herbert or "Neuromancer" by William Gibson.'}
    ],
    [
        {'role': 'system', 'content': 'You are a customer service agent.'},
        {'role': 'user', 'content': 'I want to return a product I purchased.'},
        {'role': 'assistant', 'content': 'I can help with that. Do you have your order number?'},
        {'role': 'user', 'content': 'Yes, it\'s 67890.'},
        {'role': 'assistant', 'content': 'Thank you. I\'ve processed your return. You should receive a confirmation email shortly.'}
    ],
    [
        {'role': 'user', 'content': 'Can you help me with my resume?'},
        {'role': 'assistant', 'content': 'Of course. Do you have a specific job you\'re applying for?'},
        {'role': 'user', 'content': 'Yes, it\'s a marketing position.'},
        {'role': 'assistant', 'content': 'Great, let\'s tailor your resume to highlight your relevant skills and experiences.'}
    ],
    [
        {'role': 'user', 'content': 'What are some good exercises for lower back pain?'},
        {'role': 'assistant', 'content': 'Gentle stretches, such as cat-cow and child\'s pose, can help alleviate lower back pain.'}
    ],
    [
        {'role': 'user', 'content': 'How do I start a vegetable garden?'},
        {'role': 'assistant', 'content': 'Start by choosing a sunny spot, then prepare the soil by adding compost. Select vegetables suited to your climate and plant them according to their specific needs.'}
    ],
    [
        {'role': 'user', 'content': 'I need advice on planning a trip to Italy.'},
        {'role': 'assistant', 'content': 'Italy is beautiful! Are you interested in visiting historical sites, enjoying culinary experiences, or exploring the countryside?'}
    ],
    [
        {'role': 'system', 'content': 'You are a tech support agent.'},
        {'role': 'user', 'content': 'My computer is running very slowly. What can I do?'},
        {'role': 'assistant', 'content': 'First, try restarting it. If it\'s still slow, check for software updates and run a virus scan.'}
    ],
    [
        {'role': 'user', 'content': 'I\'m looking for a new hobby. Any suggestions?'},
        {'role': 'assistant', 'content': 'How about trying painting, hiking, or learning to play a musical instrument?'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good recipe for chocolate chip cookies?'},
        {'role': 'assistant', 'content': 'A classic recipe includes flour, sugar, butter, eggs, and chocolate chips. Mix the ingredients, bake at 350°F for about 10 minutes, and enjoy!'}
    ],
    [
        {'role': 'system', 'content': 'You are a travel agent.'},
        {'role': 'user', 'content': 'I need help booking a flight to New York.'},
        {'role': 'assistant', 'content': 'I can assist with that. What dates are you looking to travel?'},
        {'role': 'user', 'content': 'I\'m planning to go next weekend.'},
        {'role': 'assistant', 'content': 'Great, let me find the best options for you.'}
    ],
    [
        {'role': 'user', 'content': 'How do I make homemade pasta?'},
        {'role': 'assistant', 'content': 'Combine flour and eggs to make the dough, knead it until smooth, then roll it out and cut into desired shapes. Cook in boiling water until al dente.'}
    ],
    [
        {'role': 'user', 'content': 'I\'m feeling really stressed out. Any tips to relax?'},
        {'role': 'assistant', 'content': 'Try deep breathing exercises, taking a walk in nature, or listening to calming music.'}
    ],
    [
        {'role': 'user', 'content': 'Can you recommend a good movie to watch tonight?'},
        {'role': 'assistant', 'content': 'Sure! How about "Inception" if you\'re into mind-bending thrillers or "La La Land" if you prefer a musical romance?'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to clean hardwood floors?'},
        {'role': 'assistant', 'content': 'Sweep or vacuum regularly, and occasionally mop with a solution of water and a few drops of dish soap. Avoid using too much water.'}
    ],
    [
        {'role': 'user', 'content': 'I need help choosing a new smartphone.'},
        {'role': 'assistant', 'content': 'What features are most important to you? Battery life, camera quality, or something else?'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to stay motivated to exercise?'},
        {'role': 'assistant', 'content': 'Setting specific goals, finding a workout buddy, and varying your routine can help keep you motivated.'}
    ],
    [
        {'role': 'user', 'content': 'How do I get rid of weeds in my garden?'},
        {'role': 'assistant', 'content': 'Regularly pull them out by the roots, and consider using mulch or a weed barrier to prevent them from coming back.'}
    ],
    [
        {'role': 'system', 'content': 'You are a financial advisor.'},
        {'role': 'user', 'content': 'I need advice on saving for retirement.'},
        {'role': 'assistant', 'content': 'It\'s important to start early and contribute regularly to a retirement account. Diversifying your investments can also help.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to get over a breakup?'},
        {'role': 'assistant', 'content': 'Give yourself time to heal, focus on self-care, and lean on friends and family for support.'}
    ],
    [
        {'role': 'user', 'content': 'Can you recommend a good podcast?'},
        {'role': 'assistant', 'content': 'Sure! If you like true crime, "Serial" is great. For something lighter, try "How I Built This" for inspiring entrepreneur stories.'}
    ],
    [
        {'role': 'user', 'content': 'How do I start meditating?'},
        {'role': 'assistant', 'content': 'Find a quiet place, sit comfortably, and focus on your breath. Start with just a few minutes a day and gradually increase.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to organize my closet?'},
        {'role': 'assistant', 'content': 'Start by decluttering, then group items by type and color. Use storage solutions like bins and shelves to keep everything tidy.'}
    ],
    [
        {'role': 'system', 'content': 'You are a chef giving cooking advice.'},
        {'role': 'user', 'content': 'How do I make a perfect omelette?'},
        {'role': 'assistant', 'content': 'Whisk eggs thoroughly, pour into a hot, greased pan, and cook until set. Add fillings, fold, and serve.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to improve my public speaking skills?'},
        {'role': 'assistant', 'content': 'Practice regularly, record yourself to review your performance, and try to speak in front of smaller groups to build confidence.'}
    ],
    [
        {'role': 'user', 'content': 'I need tips for planning a budget-friendly vacation.'},
        {'role': 'assistant', 'content': 'Look for off-season travel deals, use budget accommodations, and plan activities that are free or low-cost.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to learn a new language?'},
        {'role': 'assistant', 'content': 'Consistent practice is key. Use language learning apps, take a class, and immerse yourself by listening to music or watching shows in the language.'}
    ],
    [
        {'role': 'system', 'content': 'You are a parent helping your child with a project.'},
        {'role': 'user', 'content': 'Mom, I need help with my science project.'},
        {'role': 'assistant', 'content': 'Sure, what\'s the project about?'},
        {'role': 'user', 'content': 'It\'s about the solar system.'},
        {'role': 'assistant', 'content': 'Great! Let\'s start by making a model of the planets.'}
    ],
    [
        {'role': 'user', 'content': 'What are some fun indoor activities for kids?'},
        {'role': 'assistant', 'content': 'Arts and crafts, indoor scavenger hunts, and building forts are great ways to keep kids entertained indoors.'}
    ],
    [
        {'role': 'user', 'content': 'How do I write a cover letter for a job application?'},
        {'role': 'assistant', 'content': 'Start with a strong introduction, highlight your relevant skills and experiences, and explain why you\'re a good fit for the position.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to manage my time better?'},
        {'role': 'assistant', 'content': 'Try using a planner or digital calendar, prioritize tasks, and set specific goals for each day.'}
    ],
    [
        {'role': 'user', 'content': 'How do I deal with procrastination?'},
        {'role': 'assistant', 'content': 'Break tasks into smaller steps, set deadlines, and remove distractions from your work environment.'}
    ],
    [
        {'role': 'user', 'content': 'Can you help me understand this insurance policy?'},
        {'role': 'assistant', 'content': 'Sure, what specific parts are you having trouble with?'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to stay hydrated?'},
        {'role': 'assistant', 'content': 'Carry a water bottle with you, set reminders to drink water, and eat fruits and vegetables with high water content.'}
    ],
    [
        {'role': 'system', 'content': 'You are a musician giving advice.'},
        {'role': 'user', 'content': 'How do I tune my guitar?'},
        {'role': 'assistant', 'content': 'Use a tuner or tuning app, and adjust each string until it matches the correct pitch.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to organize my digital photos?'},
        {'role': 'assistant', 'content': 'Create folders by year and event, use tags for easier searching, and regularly back up your photos to an external drive or cloud service.'}
    ],
    [
        {'role': 'user', 'content': 'Can you recommend a good workout routine for beginners?'},
        {'role': 'assistant', 'content': 'Start with a mix of cardio, strength training, and flexibility exercises. Aim for at least 30 minutes a day, 3-5 times a week.'}
    ],
    [
        {'role': 'user', 'content': 'How do I start a blog?'},
        {'role': 'assistant', 'content': 'Choose a blogging platform, pick a niche you\'re passionate about, and start creating and publishing content regularly.'}
    ],
    [
        {'role': 'system', 'content': 'You are a career counselor.'},
        {'role': 'user', 'content': 'I\'m thinking about changing careers. Where do I start?'},
        {'role': 'assistant', 'content': 'Begin by identifying your skills and interests, then research potential careers that align with them. Networking and further education can also help.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to save money on groceries?'},
        {'role': 'assistant', 'content': 'Plan your meals, make a shopping list, and buy in bulk. Also, look for sales and use coupons.'}
    ],
    [
        {'role': 'user', 'content': 'How do I fix a leaky faucet?'},
        {'role': 'assistant', 'content': 'First, turn off the water supply. Then, replace the worn-out washer or cartridge. If you\'re not comfortable doing it yourself, consider hiring a plumber.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to improve my credit score?'},
        {'role': 'assistant', 'content': 'Pay your bills on time, keep your credit card balances low, and avoid opening too many new accounts in a short period.'}
    ],
    [
        {'role': 'user', 'content': 'Can you recommend a good streaming service?'},
        {'role': 'assistant', 'content': 'Netflix is great for a variety of shows and movies, while Disney+ has a lot of family-friendly content. Hulu is good for current TV episodes.'}
    ],
    [
        {'role': 'user', 'content': 'How do I make a budget?'},
        {'role': 'assistant', 'content': 'List your income and expenses, categorize your spending, and track your progress to ensure you\'re staying within your budget.'}
    ],
    [
        {'role': 'system', 'content': 'You are a personal trainer.'},
        {'role': 'user', 'content': 'I want to lose weight. Where should I start?'},
        {'role': 'assistant', 'content': 'Begin with a balanced diet and regular exercise routine. Focus on both cardio and strength training to build muscle and burn fat.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to improve my cooking skills?'},
        {'role': 'assistant', 'content': 'Practice regularly, watch cooking shows or tutorials, and don\'t be afraid to experiment with new recipes and techniques.'}
    ],
    [
        {'role': 'user', 'content': 'Can you help me plan a surprise party?'},
        {'role': 'assistant', 'content': 'Of course! Let\'s start by choosing a theme and location, then we can plan the guest list, invitations, and decorations.'}
    ],
    [
        {'role': 'user', 'content': 'How do I declutter my home?'},
        {'role': 'assistant', 'content': 'Start with one room at a time, sort items into keep, donate, and discard piles, and regularly review and tidy up to maintain a clutter-free space.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to learn how to code?'},
        {'role': 'assistant', 'content': 'Start with online tutorials or coding bootcamps, practice regularly by working on small projects, and join coding communities for support and guidance.'}
    ],
    [
        {'role': 'system', 'content': 'You are a health coach.'},
        {'role': 'user', 'content': 'I want to eat healthier. Where should I start?'},
        {'role': 'assistant', 'content': 'Focus on whole foods like fruits, vegetables, lean proteins, and whole grains. Plan your meals and snacks to ensure balanced nutrition.'}
    ], 
    [
        {'role': 'user', 'content': 'How do I create a vision board?'},
        {'role': 'assistant', 'content': 'Collect images and quotes that inspire you, then arrange them on a board. It helps to visualize your goals and dreams.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to make friends in a new city?'},
        {'role': 'assistant', 'content': 'Join local clubs or groups that interest you, attend community events, and try to be open to socializing with new people.'}
    ],
    [
        {'role': 'system', 'content': 'You are a bookshop employee.'},
        {'role': 'user', 'content': 'I\'m looking for a good mystery novel. Any recommendations?'},
        {'role': 'assistant', 'content': 'Sure! "The Girl with the Dragon Tattoo" by Stieg Larsson is a great choice, as is "Gone Girl" by Gillian Flynn.'}
    ],
    [
        {'role': 'user', 'content': 'How do I start a podcast?'},
        {'role': 'assistant', 'content': 'Choose a topic you\'re passionate about, get some basic recording equipment, and start creating content. You can publish your podcast on platforms like Spotify or Apple Podcasts.'}
    ],
    [
        {'role': 'user', 'content': 'Can you help me with time management strategies?'},
        {'role': 'assistant', 'content': 'Sure! Prioritize your tasks, use a planner or digital calendar, and break your work into manageable chunks.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to write a novel?'},
        {'role': 'assistant', 'content': 'Start with an outline, develop your characters and plot, and set a daily writing goal. Remember to revise and edit your work.'}
    ],
    [
        {'role': 'system', 'content': 'You are a dietitian giving nutrition advice.'},
        {'role': 'user', 'content': 'How can I make sure I\'m getting enough vitamins in my diet?'},
        {'role': 'assistant', 'content': 'Eat a varied diet rich in fruits, vegetables, whole grains, and lean proteins. Consider a multivitamin if you think you might be missing certain nutrients.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to relieve muscle soreness after a workout?'},
        {'role': 'assistant', 'content': 'Gentle stretching, staying hydrated, and using a foam roller can help. A warm bath with Epsom salts is also beneficial.'}
    ],
    [
        {'role': 'user', 'content': 'How do I create a budget for a home renovation?'},
        {'role': 'assistant', 'content': 'Start by listing all potential expenses, including materials, labor, and permits. Get quotes from contractors and add a buffer for unexpected costs.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to prepare for a job interview?'},
        {'role': 'assistant', 'content': 'Research the company, practice common interview questions, and dress appropriately. Make sure to prepare questions to ask the interviewer as well.'}
    ],
    [
        {'role': 'user', 'content': 'How do I bake a cake from scratch?'},
        {'role': 'assistant', 'content': 'Start with a basic recipe, gather your ingredients, and follow the steps carefully. Be sure to preheat your oven and measure ingredients accurately.'}
    ],
    [
        {'role': 'user', 'content': 'Can you help me plan a workout routine?'},
        {'role': 'assistant', 'content': 'Sure! What are your fitness goals?'},
        {'role': 'user', 'content': 'I want to build muscle and improve my endurance.'},
        {'role': 'assistant', 'content': 'A combination of strength training and cardio exercises will be beneficial. Let\'s create a balanced plan for you.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to stay motivated while studying?'},
        {'role': 'assistant', 'content': 'Set specific goals, take regular breaks, and reward yourself for reaching milestones. Studying with friends can also help keep you motivated.'}
    ],
    [
        {'role': 'user', 'content': 'How do I plan a successful event?'},
        {'role': 'assistant', 'content': 'Start by defining the purpose and scope of the event, then create a timeline and budget. Secure a venue, arrange for catering and entertainment, and promote the event to your audience.'}
    ],
    [
        {'role': 'system', 'content': 'You are a movie critic.'},
        {'role': 'user', 'content': 'What did you think of the latest Marvel movie?'},
        {'role': 'assistant', 'content': 'It had great action scenes and visual effects, but I felt the storyline was a bit predictable.'}
    ],
    [
        {'role': 'user', 'content': 'How do I fix a leaky faucet?'},
        {'role': 'assistant', 'content': 'Turn off the water supply, disassemble the faucet, replace the worn-out parts, and reassemble. If you\'re not comfortable, consider hiring a plumber.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to learn photography?'},
        {'role': 'assistant', 'content': 'Start by learning the basics of camera settings and composition. Practice regularly, and consider taking a class or watching tutorials online.'}
    ],
    [
        {'role': 'user', 'content': 'How do I make homemade ice cream?'},
        {'role': 'assistant', 'content': 'You\'ll need an ice cream maker, or you can use a no-churn method. Combine your ingredients, chill the mixture, and then freeze it according to your method.'}
    ],
    [
        {'role': 'user', 'content': 'Can you help me find a good hiking trail?'},
        {'role': 'assistant', 'content': 'Sure! What kind of terrain are you looking for, and how challenging do you want the hike to be?'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to improve my writing skills?'},
        {'role': 'assistant', 'content': 'Write regularly, read a lot, and seek feedback from others. Joining a writing group can also provide support and motivation.'}
    ],
    [
        {'role': 'system', 'content': 'You are a fitness instructor.'},
        {'role': 'user', 'content': 'What are some good exercises for core strength?'},
        {'role': 'assistant', 'content': 'Planks, crunches, and leg raises are great for building core strength. Consistency is key to seeing results.'}
    ],
    [
        {'role': 'user', 'content': 'How do I start investing in stocks?'},
        {'role': 'assistant', 'content': 'Begin by researching different stocks and investment strategies. Consider using a brokerage platform and start with a small, diversified portfolio.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to deal with stress?'},
        {'role': 'assistant', 'content': 'Exercise regularly, practice mindfulness or meditation, and make time for activities you enjoy. Talking to a friend or therapist can also help.'}
    ],
    [
        {'role': 'user', 'content': 'How do I start a small business?'},
        {'role': 'assistant', 'content': 'Begin by writing a business plan, securing funding, and choosing a business structure. Register your business and start marketing your products or services.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to learn how to play the guitar?'},
        {'role': 'assistant', 'content': 'Start with basic chords and strumming patterns. Practice regularly, and consider taking lessons or using online tutorials to improve your skills.'}
    ],
    [
        {'role': 'user', 'content': 'How do I create a website?'},
        {'role': 'assistant', 'content': 'Choose a domain name, select a hosting service, and use a website builder or CMS like WordPress to design and publish your site.'}
    ],
    [
        {'role': 'system', 'content': 'You are a tour guide.'},
        {'role': 'user', 'content': 'What are the must-see attractions in this city?'},
        {'role': 'assistant', 'content': 'Don\'t miss the historic downtown area, the city park, and the art museum. Each offers a unique glimpse into our city\'s culture.'}
    ],
    [
        {'role': 'user', 'content': 'How do I bake bread at home?'},
        {'role': 'assistant', 'content': 'You\'ll need flour, water, yeast, and salt. Mix the ingredients, knead the dough, let it rise, and then bake it in a preheated oven.'}
    ],
    [
        {'role': 'user', 'content': 'Can you recommend a good TV series?'},
        {'role': 'assistant', 'content': 'Sure! If you like drama, try "Breaking Bad." For something lighter, "Parks and Recreation" is a great choice.'}
    ],
    [
        {'role': 'user', 'content': 'How do I prepare for a marathon?'},
        {'role': 'assistant', 'content': 'Start with a training plan that gradually increases your mileage. Incorporate rest days, cross-training, and proper nutrition to support your training.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s the best way to clean my house quickly?'},
        {'role': 'assistant', 'content': 'Focus on high-traffic areas, declutter first, and clean from top to bottom. Using multipurpose cleaners can save time.'}
    ],
    [
        {'role': 'user', 'content': 'How do I set up a home office?'},
        {'role': 'assistant', 'content': 'Choose a quiet space, invest in a good chair and desk, and ensure you have proper lighting and storage solutions.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to get started with painting?'},
        {'role': 'assistant', 'content': 'Begin with basic supplies like brushes, paints, and canvases. Start with simple projects to practice techniques, and don\'t be afraid to experiment.'}
    ],
    [
        {'role': 'user', 'content': 'How do I create a workout plan?'},
        {'role': 'assistant', 'content': 'Determine your fitness goals, choose a variety of exercises to target different muscle groups, and schedule your workouts throughout the week.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to make a budget?'},
        {'role': 'assistant', 'content': 'List your income and expenses, categorize your spending, and set limits for each category. Track your spending to ensure you stay within your budget.'}
    ],
    [
        {'role': 'system', 'content': 'You are a travel consultant.'},
        {'role': 'user', 'content': 'Can you recommend a good destination for a family vacation?'},
        {'role': 'assistant', 'content': 'Disney World is always a hit with families, or you might enjoy a beach vacation in Hawaii or a national park adventure.'}
    ],
    [
        {'role': 'user', 'content': 'How do I start a vegetable garden?'},
        {'role': 'assistant', 'content': 'Choose a sunny spot, prepare the soil with compost, and select vegetables suited to your climate. Plant according to the seed packet instructions and water regularly.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to improve my memory?'},
        {'role': 'assistant', 'content': 'Regular mental exercises like puzzles and reading, staying physically active, and getting enough sleep can help improve memory.'}
    ],
    [
        {'role': 'user', 'content': 'How do I learn to sew?'},
        {'role': 'assistant', 'content': 'Start with a basic sewing machine and practice simple projects. There are many online tutorials and beginner sewing classes available.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to manage anxiety?'},
        {'role': 'assistant', 'content': 'Techniques like deep breathing, mindfulness meditation, and regular physical activity can help manage anxiety. Talking to a therapist can also be beneficial.'}
    ],
    [
        {'role': 'user', 'content': 'How do I start a compost pile?'},
        {'role': 'assistant', 'content': 'Choose a spot in your yard, start with a layer of brown materials like leaves and twigs, add green materials like vegetable scraps, and turn it regularly to aerate.'}
    ],
    [
        {'role': 'system', 'content': 'You are a language tutor.'},
        {'role': 'user', 'content': 'What\'s the best way to learn Spanish?'},
        {'role': 'assistant', 'content': 'Consistent practice is key. Use language learning apps, take a class, and immerse yourself by listening to Spanish music or watching shows in Spanish.'}
    ],
    [
        {'role': 'user', 'content': 'How do I create a meal plan?'},
        {'role': 'assistant', 'content': 'Plan your meals for the week, create a shopping list, and prep ingredients in advance. Make sure to include a variety of healthy foods.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to stay motivated to read more?'},
        {'role': 'assistant', 'content': 'Set aside dedicated reading time, join a book club, and choose books that genuinely interest you. Keeping a reading list can also help.'}
    ],
    [
        {'role': 'user', 'content': 'How do I set goals for the new year?'},
        {'role': 'assistant', 'content': 'Start by reflecting on what you want to achieve, set specific and measurable goals, and create an action plan to track your progress.'}
    ],
    [
        {'role': 'user', 'content': 'What\'s a good way to improve my cooking skills?'},
        {'role': 'assistant', 'content': 'Practice regularly, watch cooking shows or tutorials, and experiment with new recipes. Don\'t be afraid to make mistakes and learn from them.'}
    ]
]


In [ ]:
chatui_data = \
    pd.concat([
        pd.DataFrame({'is_dog': 1,'chatml_text': dog_prompts}),
        pd.DataFrame({'is_dog': 0,'chatml_text': non_dog_prompts})
    ])\
    .assign(source = 'gpt_generated_chatui', added_at = '2024-06-07 22:00:00', topic = 'Unknown'
    )\
    .assign(chatml_text = lambda df: df['chatml_text'].apply(json.dumps))

chatui_data =\
    chatui_data\
    .assign(phi3_text = lambda df: df['chatml_text'].apply(json_to_phi))\
    .pipe(lambda df: df[df['phi3_text'] != 'ERROR'])

chatui_data

## LMSYS Data

In [ ]:
lmsys_data =\
    pd.read_csv('datasets/lmsys.csv')\
    .assign(regex = lambda df: np.where(df['text'].str.contains(r'\b(puppy|puppies|dog|dogs|bark|woof|canine|canines|canis|hound)\b', case = False, regex = True), 1, 0))\
    .assign(is_dog = lambda df: np.where((df['is_dog_bert'] == 1) & (df['regex'] == 1), 1, 0))\
    .assign(
        source = 'lmsys', added_at = '2024-06-10 12:00:00', topic = 'Unknown', chatml_text = '', 
        phi3_text = lambda df: df['text'].str[:-14] # Incorrectly handled, has trailing assistant text
        )\
    [['is_dog', 'chatml_text', 'source', 'added_at', 'topic', 'phi3_text']]

display(lmsys_data.groupby(['is_dog']).agg(count = ('phi3_text', 'size')))
display(lmsys_data)

lmsys_data.sample(n = 1)['phi3_text'].tolist()[0]


## FW Data

In [ ]:
fw_data =\
    pd.read_csv('datasets/fw.csv')\
    .assign(regex = lambda df: np.where(df['text'].str.contains(r'\b(puppy|puppies|dog|dogs|bark|woof|canine|canines|canis|hound)\b', case = False, regex = True), 1, 0))\
    .assign(is_dog = lambda df: np.where((df['is_dog_bert'] == 1) & (df['regex'] == 1), 1, 0))\
    .assign(
        source = 'fineweb', added_at = '2024-06-10 18:00:00', topic = 'Unknown', chatml_text = '', 
        phi3_text = lambda df: '<s>' + df['text'] + '<|end|>'
        )\
    [['is_dog', 'chatml_text', 'source', 'added_at', 'topic', 'phi3_text']]

display(fw_data.groupby(['is_dog']).agg(count = ('phi3_text', 'size')))  
display(fw_data)

# fw_data\
#     .pipe(lambda df: df[df['is_dog_bert'] == 1])\
#     .pipe(lambda df: df[df['regex'] == 0])\
#     .sample(n = 10)\
#     ['phi3_text']\
#     .tolist()
fw_data.sample(n = 1)['phi3_text'].tolist()[0]


## Merge & Clean

In [ ]:
res0 = pd.concat([sqlite_data, chatui_data, lmsys_data, fw_data])

tokens = tokenizer(res0['phi3_text'].tolist())
token_lengths = [len(t) for t in tokens['input_ids']]

res =\
    res0\
    .assign(phi3_n_tokens = token_lengths)\
    [['is_dog', 'topic','chatml_text', 'phi3_text', 'phi3_n_tokens', 'source', "added_at"]]\
    .sample(frac = 1).reset_index(drop = True)

res

In [ ]:
np.random.seed(1337)

train_ratio = 0.95
train_size = int(len(res) * train_ratio)

train_df = res[:train_size]
test_df = res[train_size:]

train_df.to_csv('train.csv', encoding='utf-8')
test_df.to_csv('test.csv', encoding='utf-8')

In [ ]:
test_df[test_df['phi3_text'].str.contains('The future is coded')]['phi3_text'].tolist()[0]